In [ ]:
!pip install flask-ngrok

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import torch.optim as opt
import torch.nn.functional as F


#CUDA_VISIBLE_DEVICES=""

class RegressionTrainDataset(Dataset):

    def __init__(self):
        # Inicijalizacija ili preuzimanje podataka  
        # Čitanje .csv fajla uz pomoć numpzy
        xy = np.loadtxt('calories_tr.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.br_uzoraka = xy.shape[0]
      
        #prva kolona je ciljana vrednost, ostalo su svojstva
        self.x_data = torch.from_numpy(xy[:, 1:]) # [br_uzoraka, br_svojstava]
        self.y_data = torch.from_numpy(xy[:, [0]]) # [br_uzoraka, 1]

    # dataset[i] vraća i-ti uzorak
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # pozivom len(dataset) vraća veličinu skupa podataka
    def __len__(self):
        return self.br_uzoraka

# kreirati objekat zapredstavljanje skupa podataka obuke
train_set = RegressionTrainDataset()


class RegressionTestDataset(Dataset):

    def __init__(self):
        # Inicijalizacija ili preuzimanje podataka  
        # Čitanje .csv fajla uz pomoć numpzy
        xy = np.loadtxt('calories_tr.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.br_uzoraka = xy.shape[0]
      
        #prva kolona je ciljana vrednost, ostalo su svojstva
        self.x_data = torch.from_numpy(xy[:, 1:]) # [br_uzoraka, br_svojstava]
        self.y_data = torch.from_numpy(xy[:, [0]]) # [br_uzoraka, 1]

    # dataset[i] vraća i-ti uzorak
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # pozivom len(dataset) vraća veličinu skupa podataka
    def __len__(self):
        return self.br_uzoraka


# Čitamo ceo skup podataka uz pomoč DataLoader
# shuffle: promešati podatke - dobro za obuku 
# num_workers: brže se učitava za više procesora
train_loader = DataLoader(dataset=train_set,
                          batch_size=5,
                          shuffle=True,
                          num_workers=2)


class RegresionaMreza(torch.nn.Module):
    def __init__(self, input_size):
        super(RegresionaMreza, self).__init__()

        self.input_size = input_size

        self.linear_1 = torch.nn.Linear(input_size, 40)
        self.linear_2 = torch.nn.Linear(40, 40)
        self.linear_3 = torch.nn.Linear(40, 40)

        self.linear_4 = torch.nn.Linear(40, 1)

    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = F.relu(self.linear_2(out))
        out = F.relu(self.linear_3(out))
        out = self.linear_4(out)
        return out

mreza=RegresionaMreza(3)

optimizer=opt.Adam(mreza.parameters(), lr=0.01)
f_greske=torch.nn.MSELoss()

# Petlja za obuku
br_epoha = 20
ukupno_uzoraka = len(train_set)

for epoch in range(br_epoha):
    for i, (ulazi, izlazi) in enumerate(train_loader):
        predikcije=mreza(ulazi)
        gubici=f_greske(predikcije,izlazi)
       # print('Ocekivano', izlazi)
       # print('Predikcija', predikcije)
        gubici.backward()
        optimizer.step()
        mreza.zero_grad()

test_set = RegressionTestDataset()
test_loader = DataLoader(dataset=test_set,
                          batch_size=1,
                          shuffle=True,
                          num_workers=1)
ukupno_test=len(test_set)
print('Ukupno test', ukupno_test)
zbir=0

for i, (ulazi, izlazi) in enumerate(test_loader):
  predikcije=mreza(ulazi)
  print('Ocekivano', izlazi)
  print('Predikcija', predikcije)
  print('Greska', abs(predikcije-izlazi))
  procenat=(abs(predikcije-izlazi)/izlazi)*100 
  zbir=zbir+procenat
  print('Procentualno', procenat )
print('Duzina test', test_set.__len__())  
print('Prosecna greska ', zbir/test_set.__len__())



from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request
  
app = Flask(__name__)
run_with_ngrok(app)
  
@app.route("/regresija")
def regresija():
    tezina = float(request.args.get('tezina'))
    tip = float(request.args.get('tip'))
    trajanje = float(request.args.get('trajanje'))
    ulazi=torch.tensor([[tezina, tip, trajanje]])
    predikcije=mreza(ulazi)
    print(predikcije)
    print(predikcije[0].item())
    return str(predikcije[0].item())
  
if __name__ == "__main__":
  app.run()


Ukupno test 12
Ocekivano tensor([[590.]])
Predikcija tensor([[626.2822]], grad_fn=<AddmmBackward>)
Greska tensor([[36.2822]], grad_fn=<AbsBackward>)
Procentualno tensor([[6.1495]], grad_fn=<MulBackward0>)
Ocekivano tensor([[654.]])
Predikcija tensor([[854.4606]], grad_fn=<AddmmBackward>)
Greska tensor([[200.4606]], grad_fn=<AbsBackward>)
Procentualno tensor([[30.6515]], grad_fn=<MulBackward0>)
Ocekivano tensor([[940.]])
Predikcija tensor([[868.7838]], grad_fn=<AddmmBackward>)
Greska tensor([[71.2162]], grad_fn=<AbsBackward>)
Procentualno tensor([[7.5762]], grad_fn=<MulBackward0>)
Ocekivano tensor([[1070.]])
Predikcija tensor([[962.9199]], grad_fn=<AddmmBackward>)
Greska tensor([[107.0801]], grad_fn=<AbsBackward>)
Procentualno tensor([[10.0075]], grad_fn=<MulBackward0>)
Ocekivano tensor([[817.]])
Predikcija tensor([[861.6222]], grad_fn=<AddmmBackward>)
Greska tensor([[44.6222]], grad_fn=<AbsBackward>)
Procentualno tensor([[5.4617]], grad_fn=<MulBackward0>)
Ocekivano tensor([[931.]])
Pre

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3e59cc4d386b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Jul/2021 15:58:01] "GET /regresija?tezina=70&tip=1&trajanje=1 HTTP/1.1" 200 -


tensor([[666.1885]], grad_fn=<AddmmBackward>)
666.1885375976562


...

tensor([[666.1885]], grad_fn=<AddmmBackward>)
666.1885375976562
tensor([[666.1885]], grad_fn=<AddmmBackward>)
666.1885375976562


127.0.0.1 - - [04/Jul/2021 16:26:15] "GET /regresija?tezina=70&tip=1&trajanje=1 HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2021 16:26:16] "GET /regresija?tezina=70&tip=1&trajanje=1 HTTP/1.1" 200 -


tensor([[666.1885]], grad_fn=<AddmmBackward>)
666.1885375976562
tensor([[666.1885]], grad_fn=<AddmmBackward>)
666.1885375976562


127.0.0.1 - - [04/Jul/2021 16:26:17] "GET /regresija?tezina=70&tip=1&trajanje=1 HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2021 16:26:18] "GET /regresija?tezina=70&tip=1&trajanje=1 HTTP/1.1" 200 -


tensor([[666.1885]], grad_fn=<AddmmBackward>)
666.1885375976562
tensor([[666.1885]], grad_fn=<AddmmBackward>)
666.1885375976562
